In [1]:
import camelot
import tabula
import pandas as pd

### Die Dokumentation ist hier

Hier ist die Dokumentation von [Camelot](https://camelot-py.readthedocs.io/en/master/)

### Wir arbeiten mit den Unfällen, die Sandra interessieren

In [2]:
#Kein Resultat 
tables = camelot.read_pdf('https://www.police.be.ch/police/de/index/ueber-uns/kantonspolizei/statistik/Verkehrsstatistik.assetref/dam/documents/POM/Police/de/Statistik/Bericht_110%20ohne%20AB_AS%20Gemeinden-Stand%202018%20%28D%29.pdf', pages = "1") 
tables
#Ansteuern können wir die Tabelle auch mit der url selber:
#https://www.police.be.ch/police/de/index/ueber-uns/kantonspolizei/statistik/Verkehrsstatistik.assetref/dam/documents/POM/Police/de/Statistik/Bericht_110%20ohne%20AB_AS%20Gemeinden-Stand%202018%20%28D%29.pdf

<TableList n=0>

In [3]:
#Kein Resultat
tables = camelot.read_pdf('https://www.police.be.ch/police/de/index/ueber-uns/kantonspolizei/statistik/Verkehrsstatistik.assetref/dam/documents/POM/Police/de/Statistik/Bericht_110%20ohne%20AB_AS%20Gemeinden-Stand%202018%20%28D%29.pdf', flavor='lattice', pages = "1") 
tables

<TableList n=0>

In [4]:
#Resultat, hurrah!
tables = camelot.read_pdf('https://www.police.be.ch/police/de/index/ueber-uns/kantonspolizei/statistik/Verkehrsstatistik.assetref/dam/documents/POM/Police/de/Statistik/Bericht_110%20ohne%20AB_AS%20Gemeinden-Stand%202018%20%28D%29.pdf', flavor='stream', pages = "1") 
tables

<TableList n=1>

In [5]:
#hmm, Aber nur mit einer unvollständigen Tabelle :-/
tables[0].df

,0,1,2,3,4,5,6,7
0,nach Unfallfolgen und Gemeinde,,,,,,,
1,,,,,,davon,,
2,,,Unfälle mit,,,lebens-,,
3,,Unfälle mit,Personen-,,Schwer-,bedrohlich,erheblich,
4,,Sach-schaden,schaden,Getötete,verletzte,Verletzte¹,Verletzte¹,Leicht-verletzte
5,Total,2 489,2 397,34,506,30,476,2 344
6,Aarberg,26,12,0,2,0,2,15
7,Aarwangen,10,15,0,2,0,2,14
8,Adelboden,8,11,1,3,0,3,9
9,Aefligen,0,0,0,0,0,0,0


In [6]:
#Wir müssen dem Code sagen, wo er überall nachschauen muss. Nämlich auf der ganzen Seite.
tables_stream = camelot.read_pdf("https://www.police.be.ch/police/de/index/ueber-uns/kantonspolizei/statistik/Verkehrsstatistik.assetref/dam/documents/POM/Police/de/Statistik/Bericht_110%20ohne%20AB_AS%20Gemeinden-Stand%202018%20%28D%29.pdf", flavor='stream', pages='all', table_areas=['0,842,595,0'])

## Datenbereinigung 

In [7]:
# Nun müssen alle Tabelln zusammengeführt werden. Aber wie soll eine einzelne aussehen?
test_df = tables_stream[3].df

In [8]:
#Ab der 8 Zeile
data = test_df[9:]

In [9]:
headers = ['Gemeinde', 'Sachschaden', 'Personenschaden', 'Getötete', 
           'Schwerverletzte', 'Lebensbedrohlich Verletzte', 
           'Erheblich Verletzte', 'Leichtverletzte']

In [10]:
data.columns = headers

In [11]:
data.head()

,Gemeinde,Sachschaden,Personenschaden,Getötete,Schwerverletzte,Lebensbedrohlich Verletzte,Erheblich Verletzte,Leichtverletzte
9,Langenthal,35,46,1,7,0,7,50
10,Langnau im Emmental,17,19,2,2,0,2,17
11,Lauenen,1,0,0,0,0,0,0
12,Laupen,12,5,0,0,0,0,6
13,Lauperswil,4,10,0,2,1,1,13


### Und nun hängen wir alls zusammen

In [12]:
#eine leere df bauen
ff = pd.DataFrame()
headers = ['Gemeinde', 'Sachschaden', 'Personenschaden', 'Getötete', 
           'Schwerverletzte', 'Lebensbedrohlich Verletzte', 
           'Erheblich Verletzte', 'Leichtverletzte']

#Und nun durch alles iterieren
#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
for number in range(0,8):
    temp = tables_stream[number].df #
    temp = temp[9:]
    temp.columns = headers
    frames = [ff, temp]
    ff = pd.concat(frames)

In [13]:
ff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347 entries, 9 to 55
Data columns (total 8 columns):
Gemeinde                      347 non-null object
Sachschaden                   347 non-null object
Personenschaden               347 non-null object
Getötete                      347 non-null object
Schwerverletzte               347 non-null object
Lebensbedrohlich Verletzte    347 non-null object
Erheblich Verletzte           347 non-null object
Leichtverletzte               347 non-null object
dtypes: object(8)
memory usage: 24.4+ KB


In [14]:
## Mit leeren Strings umgehen
def clean(elem):
    if elem == "":
        return 0
    else:
        return elem
ff = ff.applymap(clean)

In [15]:
## Die Gemeinden zwischenzeitlich löschen und dann wieder reinnehmen bei der Umwandlung in Zahlen
gem = ff['Gemeinde'] 
del ff['Gemeinde']
ff = ff.astype(int)
ff['Gemeinde'] = gem
ff = ff[['Gemeinde', 'Sachschaden', 'Personenschaden', 'Getötete', 
           'Schwerverletzte', 'Lebensbedrohlich Verletzte', 
           'Erheblich Verletzte', 'Leichtverletzte']].copy() ##Reihenfolge

In [16]:
#index anpassen
ff = ff.reset_index()
del ff['index']

In [17]:
ff

,Gemeinde,Sachschaden,Personenschaden,Getötete,Schwerverletzte,Lebensbedrohlich Verletzte,Erheblich Verletzte,Leichtverletzte
0,Aarberg,26,12,0,2,0,2,15
1,Aarwangen,10,15,0,2,0,2,14
2,Adelboden,8,11,1,3,0,3,9
3,Aefligen,0,0,0,0,0,0,0
4,Aegerten,3,3,0,0,0,0,4
...,...,...,...,...,...,...,...,...
342,Zollikofen,21,15,1,4,0,4,10
343,Zuzwil (BE),0,0,0,0,0,0,0
344,Zweisimmen,6,5,1,1,1,0,4
345,Zwieselberg,1,1,0,1,0,1,0


In [18]:
ff.sort_values(by='Sachschaden', ascending=False)

,Gemeinde,Sachschaden,Personenschaden,Getötete,Schwerverletzte,Lebensbedrohlich Verletzte,Erheblich Verletzte,Leichtverletzte
23,Bern,353,345,5,69,7,62,333
25,Biel/Bienne,196,201,1,23,1,22,193
292,Thun,129,138,0,26,2,24,135
141,Köniz,78,71,0,11,1,10,64
43,Burgdorf,48,48,2,7,0,7,45
...,...,...,...,...,...,...,...,...
178,Merzligen,0,1,0,0,0,0,1
177,Melchnau,0,3,0,0,0,0,3
160,Ligerz,0,2,0,1,0,1,1
148,Landiswil,0,2,0,0,0,0,3
